# Landing AI + Ravin = :)

The above message is pretty clear to me, but let's see if we can train a Neural Net to find that message in random noise

## Dataset
The input to our neural net will be a set of images that either contain our  important message or a string of random characters. For extra difficulty the random AI will have a higher occurence of :) emojis, + and = characters as well. You will need to install the LandingAI package if following along.

### Generate dataset
Let's go ahead and generate some images. In the Landing AI package there the image class contains all the code used to generate test and train images


In [ ]:
%load_ext autoreload
%autoreload 2
from landingai import image
image.gen_images(train_examples=5000, test_examples=200, delete=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train exists, deleting directory and images and creating new ones
test exists, deleting directory and images and creating new ones


### Data Load
Each image now needs to be loaded and labeled, and the target vector needs to be one hot encoded

In [ ]:
from landingai import model_utils
x_train, y_train = model_utils.load_data("train")
x_test, y_test = model_utils.load_data("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### One Hot Encode Targets

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Train Model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, LeakyReLU
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters = 8, kernel_size=2, strides=2, padding='same', input_shape=(270,270,1)))
model.add(LeakyReLU(alpha =.1))
model.add(MaxPooling2D(pool_size=4))

model.add(Conv2D(filters = 4, kernel_size=2, strides=1, padding='valid'))
model.add(LeakyReLU(alpha =.1))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 20)

In [ ]:
model.save("Classifier.hdf5")

# Show random images and display class value

In [ ]:
import numpy as np
from IPython.core import display

def display_image(class_label, img_path):
    if class_label == 1:
        html = display.HTML('<h1 style="color:green;"> Correct class <h1>')
    else:
        html = display.HTML('<h1 style="color:red;"> Wrong class <h1>')
    
    # Convert Image to JPEG for Ipython Notebook
    i = PIL.Image.open(img_path)
    i.save("temp.jpg")
    display.display(html)
    display.display(display.Image("temp.jpg", width=270, height=270))

In [ ]:
directory = "test"
for file in os.listdir(directory)[:5]:
    img_path = os.path.join(directory, file)
    pred = model.predict(model_utils.load_image(img_path))
    class_label = np.argmax(pred)
    display_image(class_label, img_path)
    input()
    
    display.clear_output()

# Predict the real image

In [ ]:
camera_image_path = os.path.join("camera_image", "3Dprint.jpg")
camera_image = model_utils.load_camera_image(camera_image_path)
pred = model.predict(model_utils.load_image(img_path))
print(pred)
class_label = np.argmax(pred)
display_image(class_label, camera_image_path)
